In [3]:
#This block will test what value of npix is necessary for a convergent simulation

import time
import numpy as np
import matplotlib.pyplot as plt
import astropy.units as u
from astropy import constants as const
import poppy
from poppy.wfe import WavefrontError
from astropy.units import cds

fwhm_ = np.sqrt(np.log(2)/297.4178) * u.m 
lenslet_aperture = poppy.GaussianAperture(fwhm=fwhm_)
ff = 480*u.GHz # 420-540GHz
ll = const.c/ff
d_lenslet_lens = 24*u.cm

npixels=[2**11, 2**12]#, 2**13]#, 2**14]

# effective focal lengths of tertiary, secondary, primary
f_1 = 155 * u.cm
f_2 = 11940 * u.cm
f_3 = 19.199 * u.cm
tertiary_obj = poppy.QuadraticLens(f_lens=f_3)
secondary_obj = poppy.QuadraticLens(f_lens=f_2)
primary_obj = poppy.QuadraticLens(f_lens=f_1)

#Radii
coldstop = poppy.CircularAperture(radius=3.81*u.cm)
baffle = poppy.CircularAperture(radius=3.874*u.cm) 
tertiary_trunc = poppy.CircularAperture(radius=8*u.cm)
secondary_trunc = poppy.CircularAperture(radius=14.697*u.cm)
primary_trunc = poppy.CircularAperture(radius=40*u.cm)


# separations 
d_lens_3 = 33.06 * u.cm
d_3_2 = 86.562 * u.cm
d_IF = 19.2 * u.cm
d_2_1 = 104.995 * u.cm

# position of baffles above lens in inches
baffle_z_array = [0.350,
        0.700,
        1.050,
        2.150,
        2.500,
        2.850,
        3.950,
        4.300,
        4.650]

for i,baffle_z in enumerate(baffle_z_array):
    baffle_z_array[i] = (baffle_z*u.cds.inch).to(u.cm)

for npixit in npixels:  
    tic=time.time()
    
    wf = poppy.FresnelWavefront(beam_radius=8.1*u.cm, wavelength=ll, npix=npixit) 

    wf *= lenslet_aperture
    
    wf *= coldstop
    y, x = wf.coordinates()
    initial_power=np.sum(wf.intensity)

    wf.propagate_fresnel(0.35*u.cm)
        
    for i,baffle_z in enumerate(baffle_z_array):
        if i==0:
            wf.propagate_fresnel(baffle_z)
        else:
            wf.propagate_fresnel(baffle_z-baffle_z_array[i-1])
        wf*=baffle

    wf.propagate_fresnel(d_lens_3-baffle_z_array[-1]) #propagate to center of tertiary
   
    wf *= tertiary_obj
    wf *= tertiary_trunc

    wf.propagate_fresnel(d_IF) 
    
    trex = poppy.CircularAperture(radius=4*u.cm)
    wf *= trex
    
    wf.propagate_fresnel(d_3_2-d_IF)
    
    wf *= secondary_obj
    wf *= secondary_trunc
    wf.propagate_fresnel(d_2_1)

    y, x = wf.coordinates()
    
    wf *= primary_obj
    wf *= primary_trunc
    wf.propagate_fresnel(150*u.m)
    y_0, x_0 = wf.coordinates()
    z_0 = wf.intensity
    array_length = z_0.shape[0]
    half_array_length = int(z_0.shape[0]/2)

    plt.plot(x_0[half_array_length,half_array_length:],10*np.log10(z_0[half_array_length,half_array_length:]),
            label='npix='+str(npixit))
    print(time.time()-toc)
         
plt.title('Beam at 150m (closest contact by balloon)')
plt.xlabel('Distance from center (cm)')
plt.ylabel('Transmission (dB)')
#plt.xlim((0,0.15))
#plt.ylim((-2,0))
#plt.yscale('log')
plt.legend()
plt.show()
    
            


MemoryError: Unable to allocate 128. MiB for an array with shape (4096, 4096) and data type float64